<a href="https://colab.research.google.com/github/HussamCheema/DermCNN/blob/master/DermCNNPart5_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/HussamCheema/DermCNN.git

Cloning into 'DermCNN'...
remote: Enumerating objects: 11375, done.
remote: Counting objects: 100% (11375/11375), done.
remote: Compressing objects: 100% (11375/11375), done.
remote: Total 11375 (delta 18), reused 11335 (delta 0), pack-reused 0
Receiving objects: 100% (11375/11375), 438.43 MiB | 31.01 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Checking out files: 100% (11363/11363), done.


In [16]:
from __future__ import print_function, division
import os
import cv2
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np

class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 150
        self.img_cols = 150
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 7
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
      
        X_train = []
        y_train = []
        
        # Load the dataset
#         (X_train, y_train), (_, _) = mnist.load_data()

        for folder in range(1,8):
          for image in os.listdir(os.getcwd()+'/DermCNN/'+str(folder)+'/'):
            try:
              x = cv2.imread(os.getcwd()+'/DermCNN/'+str(folder)+'/'+image, cv2.IMREAD_GRAYSCALE)
              x = cv2.resize(x, (150,150))
              X_train.append(x)
              y_train.append(folder-1)
            except: pass
        X_train = np.array(X_train)
        y_train = np.array(y_train)

        y_train = keras.utils.to_categorical(y_train, 7)

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Image: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("sample_data/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=20000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_113 (Dense)            (None, 512)               11520512  
_________________________________________________________________
leaky_re_lu_85 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_114 (Dense)            (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_86 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_115 (Dense)            (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_87 (LeakyReLU)   (None, 512)               0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.694278, acc.: 21.88%] [G loss: 0.686358]
1 [D loss: 0.689584, acc.: 29.69%] [G loss: 0.682497]
2 [D loss: 0.631841, acc.: 50.00%] [G loss: 0.677981]
3 [D loss: 0.622461, acc.: 60.94%] [G loss: 0.677248]
4 [D loss: 0.593386, acc.: 59.38%] [G loss: 0.670171]
5 [D loss: 0.522999, acc.: 62.50%] [G loss: 0.650069]
6 [D loss: 0.496566, acc.: 64.06%] [G loss: 0.628236]
7 [D loss: 0.484381, acc.: 73.44%] [G loss: 0.607702]
8 [D loss: 0.453025, acc.: 90.62%] [G loss: 0.609381]
9 [D loss: 0.425322, acc.: 84.38%] [G loss: 0.585887]
10 [D loss: 0.360726, acc.: 89.06%] [G loss: 0.478906]
11 [D loss: 0.284790, acc.: 90.62%] [G loss: 0.774148]
12 [D loss: 0.313844, acc.: 89.06%] [G loss: 0.564352]
13 [D loss: 0.278446, acc.: 84.38%] [G loss: 0.786310]
14 [D loss: 0.255662, acc.: 92.19%] [G loss: 0.750951]
15 [D loss: 0.270377, acc.: 87.50%] [G loss: 0.558644]
16 [D loss: 0.284685, acc.: 87.50%] [G loss: 0.549887]
17 [D loss: 0.481801, acc.: 70.31%] [G loss: 0.940869]
18 [D loss: 0.363574

KeyboardInterrupt: ignored